# ADM-HW4: GROUP #10

#### Xavier Del Giudice


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  Valeria Avino| 1905974 |   avino.1905974@studenti.uniroma1.it                                      |
|  |  |                       |

## Import and Install required packages

In [ ]:
%pip install pandas numpy

In [2]:
import pandas as pd
import numpy as np

# 1. Recommendation System with LSH

## 1.1 Data Preparation

Reading the dataset and checking (if any) missing data.

In [23]:
#reading from our dataset
genome_scores = pd.read_csv('MovieLens/genome_scores.csv')
genome_tags = pd.read_csv('MovieLens/genome_tags.csv')
link = pd.read_csv('MovieLens/link.csv')
movie = pd.read_csv('MovieLens/movie.csv')
rating = pd.read_csv('MovieLens/rating.csv')
tags = pd.read_csv('MovieLens/tag.csv')

In [24]:
genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


In [25]:
#checking missing values
missing_values = genome_scores.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [26]:
genome_scores.duplicated().sum()

0

scores dataset has no missing values or duplicated rows

In [27]:
genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [28]:
missing_values = genome_tags.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [29]:
genome_tags.duplicated().sum()

0

In [30]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [31]:
missing_values = link.isnull().sum()
missing_values[missing_values > 0]

tmdbId    252
dtype: int64

In [32]:
link.duplicated().sum()

0

In [33]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [34]:
movie.duplicated().sum()

0

In [35]:
missing_values = movie.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [36]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [37]:
missing_values = rating.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [38]:
rating.duplicated().sum()

0

In [52]:
# converting timestamps into different columns for better readability
date_time = pd.to_datetime(rating['timestamp'])
import calendar
rating['year'] = date_time.dt.year
rating['month'] = date_time.dt.month
rating['day'] = date_time.dt.day
rating['time'] = date_time.dt.time
rating.head()

,userId,movieId,rating,timestamp,year,month,day,time
0,1,2,3.5,2005-04-02 23:53:47,2005,4,2,23:53:47
1,1,29,3.5,2005-04-02 23:31:16,2005,4,2,23:31:16
2,1,32,3.5,2005-04-02 23:33:39,2005,4,2,23:33:39
3,1,47,3.5,2005-04-02 23:32:07,2005,4,2,23:32:07
4,1,50,3.5,2005-04-02 23:29:40,2005,4,2,23:29:40


In [58]:
highest_rated = rating.loc[rating['rating'].idxmax(), 'movieId']
lowest_rated = rating.loc[rating['rating'].idxmin(), 'movieId']
#reading titles from movie dataset
print(f'The highest rated movie is {movie.loc[movie['movieId'] == highest_rated, 'title' ].values[0]} ')
print(f'The lowest rated movie is {movie.loc[movie['movieId'] == lowest_rated, 'title'].values[0]} ')

The highest rated movie is Lord of the Rings: The Fellowship of the Ring, The (2001) 
The lowest rated movie is Nemesis 2: Nebula (1995) 


In [59]:
# splitting genres column to do some stat on it
genres = movie['genres'].str.split(pat='|', expand=True).fillna(0)
max_genres_num= movie['genres'].str.split('|').map(len).max()
genres.columns = [f'genre{i+1}' for i in range(max_genres_num)]
movie = pd.concat([movie, genres], axis=1)

movie.head()

,movieId,title,genres,genre1,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure,Animation,Children,Comedy,Fantasy,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure,Children,Fantasy,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy,Romance,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy,Drama,Romance,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,Comedy,0,0,0,0,0,0,0,0,0


In [62]:
# percentage brakedown for movie genres
all_genres = movie[['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10']].values.flatten()
all_genres = all_genres[all_genres != 0]
genre_counts = pd.Series(all_genres).value_counts()
genre_percentages = round((genre_counts / genre_counts.sum()) * 100,2)

# Top 10 most common genres with their percentages
top_genres = genre_percentages.head(10)
print(top_genres)

Drama          24.53
Comedy         15.39
Thriller        7.68
Romance         7.59
Action          6.47
Crime           5.40
Horror          4.80
Documentary     4.54
Adventure       4.28
Sci-Fi          3.20
Name: count, dtype: float64


In [39]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.2+ MB


In [ ]:
missing_values = tags.isnull().sum()
missing_values[missing_values > 0]
#we could drop this rows or just keep them to check which user did not give a tag for which movies

tag    16
dtype: int64

In [41]:
tags.duplicated().sum()

0

In [ ]:
# converting timestamps into different columns for better readability
date_time = pd.to_datetime(tags['timestamp'])
import calendar
tags['year'] = date_time.dt.year
tags['month'] = date_time.dt.month
tags['day'] = date_time.dt.day
tags['time'] = date_time.dt.time

In [53]:
tags.head()

,userId,movieId,tag,timestamp,year,month,day,time
0,18,4141,Mark Waters,2009-04-24 18:19:40,2009,4,24,18:19:40
1,65,208,dark hero,2013-05-10 01:41:18,2013,5,10,01:41:18
2,65,353,dark hero,2013-05-10 01:41:19,2013,5,10,01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43,2013,5,10,01:39:43
4,65,592,dark hero,2013-05-10 01:41:18,2013,5,10,01:41:18


## 1.2 Minhash Signatures

## 1.3 Locality-Sensitive Hashing (LSH)

### 1. Bucket Creation

### 2. Query

### 3. Movie Recommendation Logic

### 4. Final Recommendation

# 2. Grouping Movies Together!

## 2.1 Feature Engineering

## 2.2 Choose your features (variables)!

## 2.3 Clustering

## 2.4 Best Algorithm

# 3. Bonus Question

# 4. Algorithmic Question